## 什么是模型蒸馏？

**模型蒸馏**，简单来说，就是用一个小模型当学生，向一个大模型老师学习。就像我们酿酒一样，把酒里的精华提炼出来，浓缩到更小的体积里。

**为什么OpenAI要这么做呢？**

因为训练和运行像GPT-4这样的大模型，成本实在是太高了！想象一下，光是为了运行一个4000亿参数的模型，就需要价值30万美元的GPU！而且大部分时间，人们可能只是问一些简单的问题，比如“草莓这个单词里有多少个字母r”。用这么强大的模型来回答这种问题，简直是大材小用。

**模型蒸馏有什么好处？**

1. **省钱！** 小模型占用的资源少，运行成本低。
2. **速度快！** 小模型的反应速度更快，不用等很久就能得到答案。
3. **保密！** OpenAI可以把小模型开源，但隐藏大模型老师的内部结构，就像可口可乐的配方一样，不会泄露核心技术。
4. **专项训练！** 模型蒸馏可以针对特定任务训练小模型，让它在某些方面比大模型老师更强。

**模型蒸馏是怎么做的？**

1. **知识提取：** 就像老师备课一样，先从大模型里提取重要的知识点。方法有很多，比如让大模型老师给数据打标签、生成更多数据、总结主题等等。
2. **知识传授：** 然后让小模型学生学习这些知识点。常用的方法包括：
    * **监督微调：** 就像老师给学生讲解题目一样，提供问题和答案让学生学习。
    * **概率分布差异最小化：** 让学生的预测结果尽可能接近老师的预测结果。
    * **强化学习：** 通过奖励和惩罚机制，引导学生学习正确的答案。
    * **排序优化：** 老师对学生的多个答案进行排序，让学生明白哪些答案好，哪些答案不好。

**模型蒸馏在哪些领域有用？**

1. **语言模型：** 可以压缩像GPT-4，o1这样的大型语言模型，让它变得更小更快，同时保留大部分理解能力。
2. **图像生成：** 可以减少图像生成模型的步骤，加快生成速度。比如“步骤蒸馏”可以把生成图片的步骤从30步减少到4步甚至1步，大大提高效率。
![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20241021191207.png)

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20241021191256.png)


模型蒸馏就像酿酒，把大模型的精华浓缩到小模型里。它可以降低成本、提高速度、保护知识产权、进行专项训练，在语言模型、图像生成等领域都有广泛应用。虽然蒸馏后的模型可能会略逊于大模型老师，但它带来的好处是巨大的。

**也许以后，我们会看到更多“蒸馏版”的AI模型，它们更便宜、更快、更易用，让更多人能够体验到AI的魅力。**

## 利用 o1 模型进行数值比较数据验证的推理蒸馏应用

如我们希望经常运行这个预测，我们希望完成速度更快、成本更低，但保持那个准确性水平。能够将 o1 的准确性提炼到 4o-mini，那就太好了，不是吗？


## 概述

数值比较在数据处理中非常常见，但由于小数点和数字位数的复杂性以及大模型的推理生成的特性，可能会导致错误。

在本教程中，我们将：

- **用o1生成一个包含正确数值比较的合成数据集/回答，作为老师的知识**。
- **我们自己校验一下o1老师的知识**。
- ----------微调 ----------
- **生成消息对作为给4o-mini的输入**。
- **将 o1 的输出提炼为 gpt-4o-mini**。
- **蒸馏模型完成**

应用拓展，如果不用o1造数值比大小的数据而是真正的业务数据呢？

https://github.com/openai/openai-cookbook/blob/3f60de050d491a92537d47b455f9155ef5e4071e/examples/o1/Using_reasoning_for_routine_generation.ipynb

## 1. 导入必要的库

在开始之前，我们需要导入一些 Python 库来帮助我们处理数据、与 OpenAI API 交互以及计算性能指标。

In [6]:
import openai  # 与 OpenAI API 交互
import json  # 处理 JSON 数据
import csv  # 读取和写入 CSV 文件
import pandas as pd  # 数据处理和分析
from sklearn.metrics import precision_score, recall_score, f1_score  # 计算性能指标
from concurrent.futures import ThreadPoolExecutor  # 并行处理

# 设置 OpenAI API 密钥
# openai.api_key = "YOUR_API_KEY"
# 指定使用的模型
MODEL = 'o1-preview'
client = openai.OpenAI()

**注意**：请确保将 `"YOUR_API_KEY"` 替换为您的实际 OpenAI API 密钥。`MODEL` 变量指定了我们将使用的模型版本。

## 2. 蒸馏知识生成

我们将定义一个函数 `generate_data()`，该函数使用 o1 的模型来生成合成数据。我们向模型提供一个详细的提示，指导它生成包含正确和错误数值比较的数据。
注意因为是测试场景，我们是让它一次回答完毕,利用了o1合成数据的能力。

另一种提供蒸馏知识的场景中，我们会存储o1的回答

```
metadata_value = "wine-distillation" # that's a funny metadata tag :-)

def call_model(model, prompt):
    response = client.chat.completions.create(
        model="o1-preview",
        store=True,
        metadata={
            "distillation": metadata_value,
        },
        messages=[
            {
                "role": "system",
                "content": "You're a sommelier expert and you know everything about wine. You answer precisely with the name of the variety/blend."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
         response_format=response_format
    )
    return json.loads(response.choices[0].message.content.strip())['variety']
```

In [65]:
def generate_data():
    prompt = """
你是一名有帮助的助手，负责生成数值比较的数据集。你的任务是创建一组数值比较，包含正确和错误的比较结果。

每条数据应包含以下字段：

- **数值1**：一个随机生成的浮点数，范围从 0 到 100，保留两位小数，数值之间的差异很小或有重复的数字在不同位置（例如 9.11 与 9.12， 67.89 和89.76）。
- **数值2**：另一个随机生成的浮点数，范围从 0 到 100，保留两位小数。
- **比较符号**：字符串，值为 `'>'`、`'<'` 或 `'='`。
- **比较结果**：一个布尔值，`True` 表示比较是正确的，`False` 表示比较是错误的。
- **问题描述**：如果比较结果为 `False`，请描述错误的原因，否则留空。

要求：

- 生成 100 条数据，其中约有 70% 的比较结果为 `True`，30% 为 `False`。
- 数值之间的差异很小，增加判断难度，但要确保比较结果是正确的。

你的回答应严格遵循有效的 CSV 格式，包含以下列：

数值1,数值2,比较符号,比较结果,问题描述

以下是一些示例数据：

数值1,数值2,比较符号,比较结果,问题描述
9.11,9.2,<,True,
15.67,15.67,=,True,
42.58,37.89,>,True,
23.45,23.46,>,False,23.45 不大于 23.46
50.00,50.00,<,False,50.00 不小于 50.00
78.90,89.01,=,False,78.90 不等于 89.01
    """
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}]
    )
    print(response.usage)
    return response.choices[0].message.content.strip()

**解释**：

- **提示设计**：我们详细地告诉模型我们需要的数据格式、字段以及如何引入错误。
- **数据比例**：通过指定 70% 的正确比较和 30% 的错误比较，我们可以测试模型在不同情况下的性能。
- **示例数据**：提供示例数据可以帮助模型理解我们期望的输出格式。

### 2.3 生成并保存数据

调用 `generate_data()` 函数，并将生成的数据保存到 CSV 文件中。

In [66]:
# 生成数据
data = generate_data()

data

CompletionUsage(completion_tokens=3604, prompt_tokens=475, total_tokens=4079, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=2048), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))


'数值1,数值2,比较符号,比较结果,问题描述\n12.34,12.56,<,True,\n78.90,78.70,>,True,\n45.67,45.67,=,True,\n23.45,23.65,<,True,\n89.01,88.99,>,True,\n56.78,56.78,=,True,\n34.56,34.55,>,True,\n67.89,67.90,<,True,\n9.12,9.11,>,True,\n50.00,50.00,=,True,\n12.34,12.33,>,True,\n78.90,78.91,<,True,\n45.67,45.66,>,True,\n23.45,23.44,>,True,\n89.01,89.02,<,True,\n56.78,56.77,>,True,\n34.56,34.57,<,True,\n67.89,67.88,>,True,\n9.12,9.13,<,True,\n50.00,50.01,<,True,\n12.34,12.35,<,True,\n78.90,78.89,>,True,\n45.67,45.68,<,True,\n23.45,23.46,<,True,\n89.01,89.00,>,True,\n56.78,56.79,<,True,\n34.56,34.56,=,True,\n67.89,67.89,=,True,\n9.12,9.12,=,True,\n50.00,50.00,=,True,\n12.34,12.34,=,True,\n78.90,78.92,<,True,\n45.67,45.65,>,True,\n23.45,23.43,>,True,\n89.01,89.03,<,True,\n56.78,56.76,>,True,\n34.56,34.58,<,True,\n67.89,67.87,>,True,\n9.12,9.14,<,True,\n50.00,49.99,>,True,\n12.34,12.36,<,True,\n78.90,78.88,>,True,\n45.67,45.69,<,True,\n23.45,23.47,<,True,\n89.01,88.99,>,True,\n56.78,56.80,<,True,\n34.56,34.54,>,Tru

In [67]:
# 将字符串写入 CSV 文件，指定编码为 utf-8-sig
with open('comparison_data.csv', 'w', encoding='utf-8-sig', newline='') as csvfile:
    csvfile.write(data)

## 3. 数据验证

### 3.1 为什么需要数据验证

在数据处理中，验证数据的准确性至关重要。对于数值比较，即使是微小的错误也可能导致重大问题。

### 3.2 定义数据验证函数

我们将定义一个函数 `validate_data(input_data)`，该函数接收一行数据，并使用模型来验证比较的正确性。

In [78]:
from pydantic import BaseModel
from typing import Optional

class ComparisonResult(BaseModel):
    # 定义属性 is_valid_right：表示比较是否有效的布尔值
    is_valid_right: bool
    # 定义属性 error_description：如果比较有效，为 None；如果无效，则提供错误描述
    error_description: Optional[str] = None

def validate_data(input_data):
    prompt = f"""
你是一名数学助手，负责验证数值比较的正确性。你将获得一条包含两个数值和比较符号的数据，你需要判断比较是否正确。

要求：

- 分析数值1、数值2和比较符号，确定比较的结果。不要包含 JSON 对象之外的任何文本或解释。

数据：

{input_data}
    """
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        response_format=ComparisonResult
    )
    response_content = response.choices[0].message.content.strip()
    try:
        result = json.loads(response_content)
        return result
    except json.JSONDecodeError:
        print(f"无法解析JSON响应：{response_content}")
        return {"is_valid": False, "issue": "无法解析模型的响应"}

**解释**：

- **提示设计**：我们指导模型只返回指定格式的 JSON 对象，以便于后续解析。
- **错误处理**：使用 `try-except` 块来捕获可能的 JSON 解析错误，确保程序的健壮性。

## 4. 加载并准备数据

### 4.1 读取 CSV 文件

使用 Pandas 读取生成的 CSV 文件。

In [103]:
import pandas as pd

# 读取 CSV 文件，假设包含标题行
data_df = pd.read_csv('comparison_data.csv')

# 将 'is_valid_right' 列转换为布尔值
# 假设 'is_valid_right' 列的值为字符串 'True' 或 'False'（不区分大小写）
data_df['is_valid_right'] = data_df['is_valid_right'].astype(str).str.lower() == 'true'

# 删除原始的第一行（假设第一行是标题后的第一行数据）
sample_df = data_df.drop(0).reset_index(drop=True)

# 定义一个函数来检查比较符号是否正确，并返回实际比较结果
def get_comparison_result(row):
    num1, num2, op = row['数值1'], row['数值2'], row['比较符号']
    if op == '>':
        return num1 > num2
    elif op == '<':
        return num1 < num2
    elif op == '=':
        return num1 == num2
    else:
        return False  # 未知的比较符号视为错误

# 应用函数，添加一列实际比较结果
sample_df['actual_comparison'] = sample_df.apply(get_comparison_result, axis=1)

# 比较 'is_valid_right' 与实际比较结果，找出不一致的条目
mismatched_df = sample_df[sample_df['is_valid_right'] != sample_df['actual_comparison']]

# 打印标记错误的比较数据
if not mismatched_df.empty:
    print("以下是 'is_valid_right' 标记错误的数据条目：")
    for idx, row in mismatched_df.iterrows():
        num1, num2, op = row['数值1'], row['数值2'], row['比较符号']
        expected = row['actual_comparison']
        # 假设 CSV 文件的标题行是第1行，数据从第2行开始
        csv_line_number = idx + 2  # 调整行号
        print(f"行号: {csv_line_number}, 数值1: {num1}, 数值2: {num2}, 比较符号: {op}, "
              f"标记的 is_valid_right: {row['is_valid_right']}, 实际比较结果: {expected}, "
              f"错误描述: {num1} {op} {num2} 的比较标记错误。")
else:
    print("所有条目的 'is_valid_right' 标记都是正确的。")



以下是 'is_valid_right' 标记错误的数据条目：
行号: 96, 数值1: 43.21, 数值2: 43.22, 比较符号: <, 标记的 is_valid_right: False, 实际比较结果: True, 错误描述: 43.21 < 43.22 的比较标记错误。
行号: 101, 数值1: 54.32, 数值2: 54.33, 比较符号: <, 标记的 is_valid_right: False, 实际比较结果: True, 错误描述: 54.32 < 54.33 的比较标记错误。


### 4.2 提取输入数据和真实标签

我们需要将数据分为输入部分（用于模型验证）和真实标签（用于评估模型性能）。可以debug查看数据格式

In [118]:
# 提取输入数据（数值1、数值2、比较符号）
input_data = data_df[['数值1', '数值2', '比较符号']].drop(0).values.tolist()

# 提取真实的比较结果和问题描述
true_is_valid = data_df['is_valid_right'].apply(lambda x: x == True).tolist()
true_issues = data_df['问题描述'].tolist()

print(true_is_valid)
print(true_issues)
print(len(true_issues))

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan

In [119]:
print(input_data)

[[78.9, 78.7, '>'], [45.67, 45.67, '='], [23.45, 23.65, '<'], [89.01, 88.99, '>'], [56.78, 56.78, '='], [34.56, 34.55, '>'], [67.89, 67.9, '<'], [9.12, 9.11, '>'], [50.0, 50.0, '='], [12.34, 12.33, '>'], [78.9, 78.91, '<'], [45.67, 45.66, '>'], [23.45, 23.44, '>'], [89.01, 89.02, '<'], [56.78, 56.77, '>'], [34.56, 34.57, '<'], [67.89, 67.88, '>'], [9.12, 9.13, '<'], [50.0, 50.01, '<'], [12.34, 12.35, '<'], [78.9, 78.89, '>'], [45.67, 45.68, '<'], [23.45, 23.46, '<'], [89.01, 89.0, '>'], [56.78, 56.79, '<'], [34.56, 34.56, '='], [67.89, 67.89, '='], [9.12, 9.12, '='], [50.0, 50.0, '='], [12.34, 12.34, '='], [78.9, 78.92, '<'], [45.67, 45.65, '>'], [23.45, 23.43, '>'], [89.01, 89.03, '<'], [56.78, 56.76, '>'], [34.56, 34.58, '<'], [67.89, 67.87, '>'], [9.12, 9.14, '<'], [50.0, 49.99, '>'], [12.34, 12.36, '<'], [78.9, 78.88, '>'], [45.67, 45.69, '<'], [23.45, 23.47, '<'], [89.01, 88.99, '>'], [56.78, 56.8, '<'], [34.56, 34.54, '>'], [67.89, 67.91, '<'], [9.12, 9.1, '>'], [50.0, 50.02, '<'

## 5. 验证数据并收集结果

### 5.1 初始化结果列表

In [120]:
# 初始化预测结果列表
pred_is_valid = []
pred_issues = []

### 5.2 定义验证每一行的函数

In [121]:
def validate_row(row):
    # 将行数据转换为字符串
    input_str = f"数值1：{row[0]}, 数值2：{row[1]}, 比较符号：{row[2]}"
    # 调用验证函数
    result = validate_data(input_str)
    # 收集结果
    pred_is_valid.append(result['is_valid_right'])
    pred_issues.append(result['error_description'])
    return result

In [122]:
validate_row(input_data[2])

{'is_valid_right': True, 'error_description': None}

### 5.3 处理数据验证


In [123]:
for i in range(len(input_data)):
    validate_row(input_data[i])

In [124]:
print(pred_is_valid)
print(pred_issues)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '78.9 is not greater than 78.88.', None, None, None, None, None, None, None, None

**解释**：

- **并行处理**：当处理大量数据时，并行处理可以显著减少处理时间。
- **线程池大小**：`max_workers` 的值可以根据您的系统资源进行调整。

## 6. 计算准确性指标

### 6.1 转换预测和真实标签为布尔值

In [125]:
# 将预测和真实的比较结果转换为布尔值
pred_is_valid_bool = [bool(val) for val in pred_is_valid]
true_is_valid_bool = [bool(val) for val in true_is_valid]

print(pred_is_valid_bool)
print(true_is_valid_bool)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False]
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, Tru

In [126]:
# 将预测和真实的比较结果转换为布尔值
pred_is_valid_bool = [bool(val) for val in pred_is_valid]
true_is_valid_bool = [bool(val) for val in true_is_valid]

print(pred_is_valid_bool)
print(true_is_valid_bool)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False]
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, Tru

### 6.2 计算精确度、召回率和 F1 分数

**1. 精确度 (Precision)**
- 定义：在模型预测为正类的样本中，真正为正类的比例
- 计算公式：TP/(TP+FP)，其中：
  - TP (True Positive): 正确预测为正类
  - FP (False Positive): 错误预测为正类
- 例子：如果模型说100个样本是正类，实际上只有80个是真正的正类，那么精确度就是80%

**2. 召回率 (Recall)**
- 定义：在所有实际为正类的样本中，模型正确预测为正类的比例
- 计算公式：TP/(TP+FN)，其中：
  - FN (False Negative): 错误预测为负类
- 例子：如果实际有100个正类样本，模型只找出了80个，那么召回率就是80%

**3. F1分数**
- 定义：精确度和召回率的调和平均数
- 计算公式：2 * (precision * recall)/(precision + recall)
- 作用：平衡精确度和召回率的综合指标


1. **互补性**
   - 精确度和召回率往往是此消彼长的关系,F1分数提供了一个平衡的评估

2. **不同场景的需求**
   - 医疗诊断：可能更注重召回率（宁可误诊也不能漏诊）
   - 垃圾邮件过滤：可能更注重精确度（宁可漏掉垃圾邮件也不误删正常邮件）

简单来说，就像用不同角度的照片来展示一个物体，这三个指标从不同维度展示了模型的性能。

In [127]:
# 计算性能指标
precision = precision_score(true_is_valid_bool, pred_is_valid_bool)
recall = recall_score(true_is_valid_bool, pred_is_valid_bool)
f1 = f1_score(true_is_valid_bool, pred_is_valid_bool)

print(f"精确度（Precision）：{precision:.2f}")
print(f"召回率（Recall）：{recall:.2f}")
print(f"F1 分数：{f1:.2f}")

精确度（Precision）：0.97
召回率（Recall）：0.96
F1 分数：0.97


In [128]:
# 找到预测不正确的索引
incorrect_indices = [i for i, (pred, true) in enumerate(zip(pred_is_valid_bool, true_is_valid_bool)) if pred != true]

# 从原始数据中提取预测不正确的行
incorrect_rows = data_df.iloc[incorrect_indices]

# 显示预测错误的行和相关信息
for idx in incorrect_indices:
    print(f"\n行号 {idx}:")
    print(f"数据: {data_df.iloc[idx].to_dict()}")
    print(f"对的结果: {true_is_valid[idx]}")
    print(f"预测结果: {pred_is_valid[idx]}")
    print(f"预测的错误描述: {pred_issues[idx]}")


行号 21:
数据: {'数值1': 78.9, '数值2': 78.89, '比较符号': '>', 'is_valid_right': True, '问题描述': nan}
对的结果: True
预测结果: False
预测的错误描述: None

行号 41:
数据: {'数值1': 78.9, '数值2': 78.88, '比较符号': '>', 'is_valid_right': True, '问题描述': nan}
对的结果: True
预测结果: False
预测的错误描述: 78.9 is not greater than 78.88.

行号 61:
数据: {'数值1': 78.9, '数值2': 78.87, '比较符号': '>', 'is_valid_right': True, '问题描述': nan}
对的结果: True
预测结果: False
预测的错误描述: 78.9 is not greater than 78.87

行号 95:
数据: {'数值1': 43.21, '数值2': 43.22, '比较符号': '<', 'is_valid_right': False, '问题描述': '43.21 不小于 43.22'}
对的结果: False
预测结果: True
预测的错误描述: None

行号 100:
数据: {'数值1': 54.32, '数值2': 54.33, '比较符号': '<', 'is_valid_right': False, '问题描述': '54.32 不大于 54.33'}
对的结果: False
预测结果: True
预测的错误描述: None





### 生成微调的消息对



In [ ]:
{"messages": [{"role": "user", "content": "分析数值1、数值2和比较符号，确定比较的结果。- 仅返回一个 JSON 对象，包含以下属性： - `is_valid_right`：一个布尔值，`true` 表示比较正确，`false` 表示比较错误。- `error_description`：如果 ` is_valid_right` 为 `false`，提供一个简短的错误描述；如果 `is_valid_right` 为 `true`，将 `error_description` 设置为 `null`。数据：数值1：9.11, 数值2：9.2, 比较符号：<"}, {"role": "assistant", "content": "{'is_valid_right': True, 'error_description': None}"}]}


In [138]:
import pandas as pd
import json

# 读取 CSV 文件，假设包含标题行
data_df = pd.read_csv('comparison_data.csv')

# 将 'is_valid_right' 列转换为布尔值
data_df['is_valid_right'] = data_df['is_valid_right'].astype(str).str.lower() == 'true'

# 随机选择100行，排除原始的第一行并重置索引
sample_df = data_df.drop(0).sample(n=100, random_state=42).reset_index(drop=True)

# 定义一个函数来进行比较并生成响应内容
def generate_message_pair(row):
    num1 = row['数值1']
    num2 = row['数值2']
    operator = row['比较符号']
    is_valid_mark = row['is_valid_right']
    error_description=row['error_description']
    
    
    # 进行实际比较
    if operator == '>':
        actual_result = num1 > num2
    elif operator == '<':
        actual_result = num1 < num2
    elif operator == '=':
        actual_result = num1 == num2
    else:
        actual_result = False  # 未知的比较符号视为错误
    
    # 确定 'is_valid_right' 是否正确
    is_valid_right = actual_result
    
    # 根据比较结果生成 error_description
    if is_valid_right:
        error_description = None
    else:
        if operator == '>':
            expected_operator = '>' if num1 > num2 else '<' if num1 < num2 else '='
        elif operator == '<':
            expected_operator = '<' if num1 < num2 else '>' if num1 > num2 else '='
        elif operator == '=':
            expected_operator = '='
        error_description = row["error_description"]
    
    # 构建用户消息内容
    user_content = (
        "分析数值1、数值2和比较符号，确定比较的结果。"
        "- 仅返回一个 JSON 对象，包含以下属性："
        " - `is_valid_right`：一个布尔值，`true` 表示比较正确，`false` 表示比较错误。"
        "- `error_description`：如果 `is_valid_right` 为 `false`，提供一个简短的错误描述；"
        " 如果 `is_valid_right` 为 `true`，将 `error_description` 设置为 `null`。"
        f"数据：数值1：{num1}, 数值2：{num2}, 比较符号：{operator}"
    )
    
    # 构建助手回复内容
    assistant_content = {
        'is_valid_right': is_valid_right,
        'error_description': error_description
    }
    
    # 构建消息对
    messages = [
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": json.dumps(assistant_content, ensure_ascii=False)}
    ]
    
    return {"messages": messages}

# 生成所有消息对
message_pairs = sample_df.apply(generate_message_pair, axis=1).tolist()

# 将所有消息对保存到一个 JSON 文件中
with open('message_pairs.jsonl', 'w', encoding='utf-8') as f:
    for message_pair in message_pairs:
        json_str = json.dumps(message_pair, ensure_ascii=False)
        f.write(json_str + '\n')

print("消息对已保存到 'message_pairs.jsonl'。")


消息对已保存到 'message_pairs.jsonl'。


然后，将数据保存为 `.jsonl` 文件，每行是一个训练示例对话。

除了训练数据，我们还可以**可选地**提供验证数据，以确保模型不会过拟合您的训练集。

一旦验证准备了数据，文件需要通过文件 API上传，以便用于微调作业：



In [139]:
training_file_name = "message_pairs.jsonl"
def upload_file(file_name: str, purpose: str) -> str:
    with open(file_name, "rb") as file_fd:
        response = client.files.create(file=file_fd, purpose=purpose)
    return response.id

# 上传训练文件
training_file_id = upload_file(training_file_name, "fine-tune")

print("训练文件 ID:", training_file_id)


训练文件 ID: file-mCYinwnAL1r6f6ij6yEYMw9L


---

## 微调

在确保您拥有正确数量和结构的数据集，并已上传文件后，下一步是创建一个微调作业。我们支持通过微调 UI或以编程方式创建微调作业。

现在，我们可以使用生成的文件和一个可选的后缀来创建微调作业，以便识别模型。响应将包含一个 `id`，您可以使用它来检索作业的更新。

**注意**：文件需要先被我们的系统处理，您可能会收到“File not ready”的错误。在这种情况下，请稍等几分钟后重试。

In [140]:
MODEL = "gpt-4o-mini-2024-07-18"

response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model=MODEL,
    suffix="recipe-ner",
)

job_id = response.id

print("作业 ID:", response.id)
print("状态:", response.status)

作业 ID: ftjob-U2atDzpwbgzgbhx8KN9CTUVk
状态: validating_files


#### 检查作业状态

您可以对 `https://api.openai.com/v1/alpha/fine-tunes` 端点发出 `GET` 请求，列出您的微调作业。在此实例中，您需要检查从上一步获得的 ID 是否最终变为 `status: succeeded`。

一旦完成，您可以使用 `result_files` 从验证集（如果您上传了验证集）中获取结果，并使用 `fine_tuned_model` 参数中的 ID 来调用您的微调模型。

In [141]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("作业 ID:", response.id)
print("状态:", response.status)
print("已训练的标记数:", response.trained_tokens)

作业 ID: ftjob-U2atDzpwbgzgbhx8KN9CTUVk
状态: running
已训练的标记数: None


您可以通过 `events` 端点跟踪微调的进度。您可以多次重新运行下面的代码单元，直到微调完成。

In [142]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-U2atDzpwbgzgbhx8KN9CTUVk
Validating training file: file-mCYinwnAL1r6f6ij6yEYMw9L
Files validated, moving job to queued state
Fine-tuning job started


In [143]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 285/300: training loss=0.00
Step 286/300: training loss=0.00
Step 287/300: training loss=0.00
Step 288/300: training loss=0.00
Step 289/300: training loss=0.00
Step 290/300: training loss=0.00
Step 291/300: training loss=0.00
Step 292/300: training loss=0.00
Step 293/300: training loss=0.00
Step 294/300: training loss=0.00
Step 295/300: training loss=0.00
Step 296/300: training loss=0.00
Step 297/300: training loss=0.00
Step 298/300: training loss=0.00
Step 299/300: training loss=0.00
Step 300/300: training loss=0.00
Checkpoint created at step 100
Checkpoint created at step 200
New fine-tuned model created
The job has successfully completed


现在微调完成，我们可以从作业中获取微调模型的 ID：

In [144]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None:
    raise RuntimeError("未找到微调模型 ID。您的作业可能尚未完成。")

print("微调模型 ID:", fine_tuned_model_id)

微调模型 ID: ft:gpt-4o-mini-2024-07-18:acmr:recipe-ner:ANEqlMx7


In [145]:
from openai import OpenAI
# 构建 OpenAI 客户端对象的实例
client = OpenAI()

models = client.models.list()
models

SyncPage[Model](data=[Model(id='gpt-4-turbo', created=1712361441, object='model', owned_by='system'), Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'), Model(id='tts-1', created=1681940951, object='model', owned_by='openai-internal'), Model(id='tts-1-1106', created=1699053241, object='model', owned_by='system'), Model(id='chatgpt-4o-latest', created=1723515131, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'), Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'), Model(id='gpt-3.5-turbo-0125', created=1706048358, object='model', owned_by='system'), Model(id='babbage-002', created=1692634615, object='model', owned_by='system'), Model(id='davinci-002', created=1692634301, object='mod

In [146]:
for model in models:
    print(model.id)

gpt-4-turbo
gpt-4-turbo-2024-04-09
tts-1
tts-1-1106
chatgpt-4o-latest
dall-e-2
whisper-1
gpt-3.5-turbo-instruct
gpt-3.5-turbo
gpt-3.5-turbo-0125
babbage-002
davinci-002
gpt-4o-mini-2024-07-18
dall-e-3
gpt-4o-mini
gpt-3.5-turbo-0301
gpt-4o-2024-05-13
o1-preview
gpt-4o-audio-preview-2024-10-01
o1-mini-2024-09-12
gpt-4o-audio-preview
tts-1-hd
tts-1-hd-1106
o1-preview-2024-09-12
o1-mini
gpt-4-1106-preview
text-embedding-ada-002
gpt-3.5-turbo-16k
text-embedding-3-small
text-embedding-3-large
gpt-4o-realtime-preview-2024-10-01
gpt-4o-realtime-preview
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-1106
gpt-3.5-turbo-0613
gpt-4-0613
gpt-4-turbo-preview
gpt-4-0125-preview
gpt-4
gpt-3.5-turbo-instruct-0914
gpt-4o-2024-08-06
gpt-4o
ft:gpt-3.5-turbo-0613:acmr:recipe-ner:7rOJnQow
ft:babbage-002:acmr::8LjwL7pi
ft:gpt-3.5-turbo-1106:acmr::8NFcWDKd
ft:gpt-3.5-turbo-1106:acmr::8NG1DFJy
ft:gpt-3.5-turbo-1106:acmr::8NGu7dB7
ft:gpt-3.5-turbo-1106:acmr::8NGecRZn
ft:gpt-3.5-turbo-1106:acmr::8NGoWxGp
ft:gpt-3.5-turbo-

---

## 推理

最后一步是使用您的微调模型进行推理。与经典的 `FineTuning` 类似，您只需在 `model` 参数中填写新的微调模型名称，调用 `ChatCompletions` 即可。

调用微调模型进行推理：

In [ ]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response.choices[0].message.content)

应用拓展，如果不用o1造数值比大小的数据而是真正的业务数据呢？

- 提高产生所需输出的可靠性
- 修正未能遵循复杂提示的错误
- 以特定方式处理许多边缘情况
- 执行一种新的技能或任务，这很难在提示中表达

https://github.com/openai/openai-cookbook/blob/3f60de050d491a92537d47b455f9155ef5e4071e/examples/o1/Using_reasoning_for_routine_generation.ipynb